In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader

from PIL import Image
import torchvision
from torchvision import datasets, models, transforms
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
from sklearn.metrics import *
import time
import os
from torch.utils import data
import random

In [ ]:
if torch.cuda.is_available():
    device = torch.device('cuda:0') 
else:
    device = torch.device('cpu')

In [ ]:
# root directory
andrea_dir = "/home/andreasabo/Documents/HNProject/"

# data directory on current machine: abhishekmoturu, andreasabo, denizjafari, navidkorhani
data_dir = "/home/nkorhani/Documents/HNProject/all_label_img/"

# read target df
csv_path = os.path.join(andrea_dir, "all_splits_1000000.csv")
data_df = pd.read_csv(csv_path, usecols=['subj_id', 'image_ids', 'view_label', 'view_train'])

In [ ]:
label_mapping = {'Other':0, 'Saggital_Right':1, 'Transverse_Right':2, 
                 'Saggital_Left':3, 'Transverse_Left':4, 'Bladder':5}

data_df['view_label'] = data_df['view_label'].map(label_mapping)

train_df = data_df[data_df.view_train == 1]
test_df = data_df[data_df.view_train == 0]

labels = {}
train_and_valid_subj_ids = []
train_and_valid_image_ids = []
test_ids = []

for ind, row in train_df.iterrows():
    train_and_valid_subj_ids.append(row['subj_id'])
    train_and_valid_image_ids.append(row['image_ids'])
    labels[row['image_ids']] = row['view_label']

for ind, row in test_df.iterrows():
    test_ids.append(row['image_ids'])
    labels[row['image_ids']] = row['view_label']

s = set()
t_v_ids = pd.DataFrame(list(zip(train_and_valid_subj_ids, train_and_valid_image_ids)), columns=['subj_ids', 'image_ids'])
id_groups = [t_v_ids for _, t_v_ids in t_v_ids.groupby('subj_ids')]
random.shuffle(id_groups)
id_groups = pd.concat(id_groups).reset_index(drop=True)
train_val_split = int(0.8*len(set(id_groups['subj_ids'].values)))
train_val_set = [i for i in id_groups['subj_ids'].values if not (i in s or s.add(i))]
cutoff = train_val_set[train_val_split]
train_portion = (id_groups['subj_ids'].values == cutoff).argmax()

train_ids = id_groups[:train_portion]['image_ids'].tolist()
valid_ids = id_groups[train_portion:]['image_ids'].tolist()

partition = {'train':train_ids, 'valid':valid_ids, 'test':test_ids}

In [ ]:
# Data augmentation and normalization for training

# Parameters
params = {'batch_size': batch_size,
          'shuffle': True,
          'num_workers': 6}

# Generators
training_set = Dataset(partition['train'], labels)
train_loader = data.DataLoader(training_set, **params)

validation_set = Dataset(partition['valid'], labels)
validation_loader = data.DataLoader(validation_set, **params)

test_set = Dataset(partition['valid'], labels)
validation_loader = data.DataLoader(test_set, **params)

dataloaders_dict = {'train':training_generator, 'val':validation_generator}

In [ ]:
class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()
        
        hidden_dim = 800
        latent_dim = 100
        self.fc1 = nn.Linear(65536, hidden_dim)
        self.fc21 = nn.Linear(hidden_dim, latent_dim)
        self.fc22 = nn.Linear(hidden_dim, latent_dim)
        self.fc3 = nn.Linear(latent_dim, hidden_dim)
        self.fc4 = nn.Linear(hidden_dim, 65536)

    def encode(self, x):
        h1 = F.relu(self.fc1(x))
        return self.fc21(h1), self.fc22(h1)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        return mu + eps*std

    def decode(self, z):
        #print("z.size() =", z.size())
        h3 = F.relu(self.fc3(z))
        #print("h3.size() =", h3.size())
        return torch.sigmoid(self.fc4(h3))

    def forward(self, x):
        mu, logvar = self.encode(x.view(-1, 65536))
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar

vae_model = VAE().to(device)
checkpoint = torch.load('results/h800_l100_e30/vae_model.pt')
vae_model.load_state_dict(checkpoint)
vae_model.eval()

for params in vae_model.parameters():
    params.requires_grad = False

In [ ]:
class Dataset(data.Dataset):
  'Characterizes a dataset for PyTorch'
  def __init__(self, list_IDs, labels):
        'Initialization'
        self.labels = labels
        self.list_IDs = list_IDs

  def __len__(self):
        'Denotes the total number of samples'
        return len(self.list_IDs)

  def __getitem__(self, index):
        'Generates one sample of data'
        # Select sample
        ID = self.list_IDs[index]

        # Load data and get label
        img_path = data_dir + ID + '.jpg'
        image = Image.open(img_path)#.convert('L')
        image = ToTensor()(image)
        
        mu, logvar = vae_model.encode(image.view(-1, 65536))
        z = vae_model.reparameterize(mu, logvar)

        y = torch.FloatTensor(self.labels[ID])

        return z, y

In [ ]:
class DeepClassifier(nn.Module):
    def __init__(self):
        super(DeepClassifier, self).__init__()
        
        self.fc1 = nn.Linear(100, 100)
        self.fc2 = nn.Linear(100, 50)
        self.fc3 = nn.Linear(50, 6)
        self.softmax = nn.Softmax()
    
    
    def forward(self, z):
        h1 = F.relu(self.fc1(z))
        h2 = F.relu(self.fc2(h1))
        out = self.softmax(self.fc3(h2))
        
        return out
    

In [ ]:
#hyperparams

epochs = 30

val_check_interval = 40

log_interval = 10

In [ ]:
classifier_model = DeepClassifier().to(device)

optimizer = optim.Adam(classifier_model.parameters(), lr=1e-4)

criterion = nn.CrossEntropyLoss()


train_loss_array = np.array([])
val_loss_array = np.array([])
test_loss_array = np.array([])




In [ ]:
def train(epoch):
    global batches_loss_array
    global train_loss_array
    
    classifier_model.train()
    train_loss = 0
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        preds = classifier_model(inputs)
        
        loss = criterion(preds, targets)
        train_loss += loss.item()
        loss.backward()
        
        optimizer.step()
        
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.2f} '.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader),
                loss.item()))
            batches_loss_array = np.append(batches_loss_array,loss.item())
            
            
        avg_train_loss = train_loss / len(train_loader.dataset) * len(inputs)
    
    #print('====> Epoch: {} Average Training loss: {:.2f}'.format(
    #      epoch, avg_train_loss))
    
    train_loss_array = np.append(train_loss_array, avg_train_loss)
    
    return avg_train_loss


def evaluation(epoch):
    global least_error
    global val_loss_array
    
    classifier_model.eval()
    eval_loss = 0
    with torch.no_grad():
        for inputs, targets in validation_loader:

            inputs, targets = inputs.to(device), targets.to(device)
            preds = classifier_model(inputs)

            loss = criterion(preds, targets)
            eval_loss += loss.item()

            avg_val_loss = eval_loss / len(eval_loader.dataset) * len(inputs)
        
    val_loss_array = np.append(val_loss_array, avg_val_loss)
    
    if least_error==-1 or least_error>avg_val_loss:
        PATH = os.path.join('saved_models', 'vae_classifier.pt')
    
        torch.save(model.state_dict(), PATH)
        
        least_error = avg_val_loss
        
    return avg_val_loss
        
def test(epoch):

    #global test_loss_array
    
    model.eval()
    test_loss = 0
    
    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            
            preds = model(inputs)
            
            test_loss += citerion(preds, targets).item()

    test_loss /= len(test_loader.dataset) * len(inputs)
    
    #test_loss_array = np.append(test_loss_array, test_loss)
    
    return test_loss


In [ ]:
for epoch in epochs:
    train_loss = train()
    val_loss = evaluation()
    
    print('====> Epoch: {}  Avg Train loss: {:.2f} ... Avg Validation loss: {:.2f}'.format(
              epoch, train_loss, val_loss))
    
test_loss = test()

print("Test Loss: {}".format(test_loss))

np.save('vae_class_train.npy', train_loss_array)
np.save('vae_class_val.npy', val_loss_array)
